In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
tr = pd.read_csv("../data/public_train.csv")
te = pd.read_csv("../data/public_test.csv")
tr['train_or_test']='train'
te['train_or_test']='test'
alldf = pd.concat([tr,te], sort=False)
alldf = alldf.rename({'PAY_MONTH_0':'PAY_MONTH_1'}, axis='columns')
alldf.shape

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(50000, 67)

# New attributes

In [3]:
agecuts=[0,30,50,200]
categories = [0,1,2]
alldf['AGE_RANGE'] = pd.cut(alldf['AGE'], agecuts, labels=categories).values.astype(int)

# Treat missing data

In [4]:
for att in alldf.columns:
    if att=="TARGET":
        pass
    elif alldf[att].dtype==object:
        alldf[att]=alldf[att].fillna("nincs_adat")
    else:
        #median of the same age range is used for the missing data
        alldf[att]=alldf[att].fillna(alldf.groupby(['AGE_RANGE'])[att].transform(np.nanmedian))

# Categorical to numerical

In [5]:
def cat_to_num(indf,attrToConvert):
    df=indf.copy()
    for attr in attrToConvert: 
        df[attr] = df[attr].astype('category').cat.codes
    return df

#use it
attrToConvert = ['BIRTH_STATE','BIRTH_CITY','RES_STATE','RES_CITY','RES_BOROUGH',
                'PROF_STATE','PROF_CITY','PROF_BOROUGH','MATE_PROF_CODE']
alldf = cat_to_num(alldf,attrToConvert)

alldf['RES_PROF_STATE'] = alldf.apply(lambda x: int(x['RES_STATE']==x['PROF_STATE']),axis=1)

alldf = alldf.drop(attrToConvert,axis=1)

In [6]:
def limitalt_dummy_valtozo(indf,oszlop,limit,drop_last=True):
    df=indf.copy()
    egyedi_elemszam=df[oszlop].nunique()
    if drop_last is True and egyedi_elemszam<=limit:
        limit = egyedi_elemszam-1
    a=df[oszlop].value_counts()[:limit]
    for value in list(a.index):
        def f(x):
            if x==value:
                return 1
            else:
                return 0
        df[oszlop+"="+str(value)]=df[oszlop].apply(f)
    return df

#use it
attrToConvert = ['APP_SUB_TYPE','M_STATUS','RES_TYPE','OCC_TYPE','EDU_LEVEL']

for attr in attrToConvert:
    alldf = limitalt_dummy_valtozo(alldf,attr,6,True)
alldf = alldf.drop(attrToConvert,axis=1)

In [7]:
#set binomial attributes
alldf['SEX'] = alldf['SEX'].apply(lambda x: 0 if x=='male' else(1 if x=='female' else 0)) 
alldf['HOME_PHONE_FLAG'] = alldf['HOME_PHONE_FLAG'].apply(lambda x: 0 if x=='no' else(1 if x=='yes' else 0))
alldf['C_NAME_GIVEN'] = alldf['C_NAME_GIVEN'].apply(lambda x: 0 if x=='no' else(1 if x=='yes' else 0))       
alldf['PROF_PHONE_GIVEN'] = alldf['PROF_PHONE_GIVEN'].apply(lambda x: 0 if x=='No' else(1 if x=='Yes' else 0)) 
alldf['P_ADDRESS_TYPE'] = alldf['P_ADDRESS_TYPE'].apply(lambda x: 0 if x=='A' else(1 if x=='B' else 0))

# Treat outliers

In [8]:
alldf['NUMBER_OF_DEPENDANTS'] = alldf['NUMBER_OF_DEPENDANTS'].apply(lambda x: x if x<=10 else 11)
alldf['M_IN_THE_JOB'] = alldf['M_IN_THE_JOB'].apply(lambda x: x if x<=1 else 2)

In [9]:
def set_max_value(indf, attributes, quanti):
    df=indf.copy()
    if type(attributes) != list:
        attributes = [attributes]
    for attr in attributes:
        quantile = df[attr].quantile(quanti)
        df[attr] = df[attr].apply(lambda x: x if x<=quantile else quantile)
    return df

#set outliers to the 0.95 quantile
attr = ['MONTHS_IN_RES']
alldf = set_max_value(alldf, attr, 0.95)

# New attributes 2

In [10]:
#Log10 of money types
attrs = ['P_MONTHLY_INCOME', 'O_INCOMES', 'P_ASSETS_VALUE', 'L_BALANCE', ]
for attr in attrs:
    alldf[attr+'log10'] = alldf[attr].apply(lambda x: np.log(x) if x>0 else ( -np.log(-x) if x!=0 else 0))
    
alldf['AGE_RANGE_AVERAGE_INCOMElog10'] = alldf.groupby(['AGE_RANGE'])['P_MONTHLY_INCOMElog10'].transform('mean')
alldf['DEV_FROM_AGE_RANGE_AVERAGE_INCOMElog10'] = alldf['AGE_RANGE_AVERAGE_INCOMElog10']-alldf['P_MONTHLY_INCOMElog10']
alldf['ALL_INCOME_PER_HEADlog10'] = alldf.apply(lambda x: (np.log10(x['P_MONTHLY_INCOME']+x['O_INCOMES'])/(x['NUMBER_OF_DEPENDANTS']+1)),axis=1)

alldf.drop('AGE_RANGE_AVERAGE_INCOMElog10',axis=1,inplace=True)
alldf.drop(attrs,axis=1,inplace=True)

In [11]:
inattr = ['PAY_MONTH_1','PAY_MONTH_2','PAY_MONTH_3','PAY_MONTH_4','PAY_MONTH_5','PAY_MONTH_6']
alldf['PAY_MONTH_AVERAGE'] = alldf[inattr].mean(axis=1)
alldf['PAY_MONTH_MIN'] = alldf[inattr].min(axis=1)
alldf['PAY_MONTH_MAX'] = alldf[inattr].max(axis=1)
alldf['PAY_MONTH_MAX'] = alldf[inattr].sum(axis=1)

inattr = ['PAY_AMOUNT_MONTH_1','PAY_AMOUNT_MONTH_2','PAY_AMOUNT_MONTH_3','PAY_AMOUNT_MONTH_4','PAY_AMOUNT_MONTH_5','PAY_AMOUNT_MONTH_6']
alldf['PAY_AMOUNT_AVERAGE'] = alldf[inattr].mean(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0))
alldf['PAY_AMOUNT_MIN'] = alldf[inattr].min(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0))
alldf['PAY_AMOUNT_MAX'] = alldf[inattr].max(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0))
alldf['PAY_AMOUNT_STD'] = alldf[inattr].std(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0))
alldf['PAY_AMOUNT_SUM'] = alldf[inattr].sum(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0)) 
alldf['MAX_PAY_AMOUNT_MONTH'] = alldf[inattr].idxmax(axis=1).apply(lambda x: int(x[-1]))
alldf['MIN_PAY_AMOUNT_MONTH'] = alldf[inattr].idxmin(axis=1).apply(lambda x: int(x[-1]))


inattr = ['BILL_AMOUNT_MONTH_1','BILL_AMOUNT_MONTH_2','BILL_AMOUNT_MONTH_3','BILL_AMOUNT_MONTH_4','BILL_AMOUNT_MONTH_5','BILL_AMOUNT_MONTH_6']
alldf['BILL_AMOUNT_AVERAGE'] = alldf[inattr].mean(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0)) 
alldf['BILL_AMOUNT_MIN'] = alldf[inattr].min(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0))
alldf['BILL_AMOUNT_MAX'] = alldf[inattr].max(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0))
alldf['BILL_AMOUNT_STD'] = alldf[inattr].std(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0))
alldf['BILL_AMOUNT_SUM'] = alldf[inattr].sum(axis=1).apply(lambda x: np.log10(x) if x>0 else ( -np.log(-x) if x!=0 else 0)) 
alldf['MAX_BILL_AMOUNT_MONTH'] = alldf[inattr].idxmax(axis=1).apply(lambda x: int(x[-1]))
alldf['MIN_BILL_AMOUNT_MONTH'] = alldf[inattr].idxmin(axis=1).apply(lambda x: int(x[-1]))

alldf['BILL_MINUS_PAY_AVERAGE'] = alldf.apply(lambda x: x['BILL_AMOUNT_AVERAGE']-x['PAY_AMOUNT_AVERAGE'],axis=1)
alldf['BILL_MINUS_PAY_MIN'] = alldf.apply(lambda x: x['BILL_AMOUNT_MIN']-x['PAY_AMOUNT_MIN'],axis=1)
alldf['BILL_MINUS_PAY_MAX'] = alldf.apply(lambda x: x['BILL_AMOUNT_MAX']-x['PAY_AMOUNT_MAX'],axis=1)
alldf['BILL_MINUS_PAY_SUM'] = alldf.apply(lambda x: x['BILL_AMOUNT_SUM']-x['PAY_AMOUNT_SUM'],axis=1)


In [12]:
alldf.shape

(50000, 99)

In [13]:
alldf.to_csv('../data/prep_df.csv', index=False)